# 청와대 청원 크롤링 코드 

## 작성자: 박하람
## 작성일자: 2020년 8월 10일 

분야별 청원 전체 카테고리 선택 후, 추천 순 정렬하여 모든 청원 가져오기

In [1]:
import sys
sys.path.append("/usr/local/lib/python3.8/site-packages")

In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
import time
import random
import re

In [3]:
def get_peti_df(start, end): 
    
    category = []
    titles = []
    address = []
    end_date = []
    consent_num = []
    
    for n in tqdm(range(start, end+1)): 
        url = f'https://www1.president.go.kr/petitions/?c=0&only=2&page={n}&order=2'
        soup = BeautifulSoup(urlopen(url), "html.parser")
            
        contents = soup.find_all('div', class_='bl_body')
        
        for i in range(7):
            peti_list = contents[0].find_all('div', class_='bl_wrap')[i]
            
            # 청원 카테고리
            category.append(peti_list.find('div', class_='bl_category').get_text()) 
            # 청원 제목 
            titles_raw = peti_list.find('div', class_='bl_subject').get_text()
            titles_split = re.split('\r|\t|\n|\xa0|제목', titles_raw)
            titles.append(''.join(titles_split))
            # 청원 주소 
            add_url = peti_list.find('a')['href']
            address.append('https://www1.president.go.kr'+add_url)
            # 청원 만료일 
            end_date.append(peti_list.find('div', class_='bl_date').get_text())
            # 청원 동의 수 
            consent_num_raw = peti_list.find('div', class_='bl_agree').get_text()
            consent_split = re.split('\r|\t|\n|\xa0', consent_num_raw)
            consent_num.append(''.join(consent_split).split()[1][:-1])
        
    petition_dict = {'카테고리':category, '제목':titles, '청원종료일': end_date, 
                 '참여인원': consent_num,'URL': address}
    df = pd.DataFrame(petition_dict)
    return df 
        

In [4]:
%time peti_df_1 = get_peti_df(1,50)

100%|██████████| 50/50 [00:14<00:00,  3.52it/s]

CPU times: user 3.46 s, sys: 148 ms, total: 3.6 s
Wall time: 14.2 s


In [5]:
%time peti_df_2 = get_peti_df(51,100)

  8%|▊         | 4/50 [00:01<00:11,  3.93it/s]


IndexError: list index out of range

In [6]:
%time peti_df_3 = get_peti_df(101,150)

  0%|          | 0/50 [00:00<?, ?it/s]


IndexError: list index out of range

In [7]:
%time peti_df_4 = get_peti_df(151,200)

  0%|          | 0/50 [00:00<?, ?it/s]


IndexError: list index out of range

In [8]:
%time peti_df_5 = get_peti_df(201,250)

  0%|          | 0/50 [00:00<?, ?it/s]


IndexError: list index out of range

In [9]:
%time peti_df_6 = get_peti_df(251, 300)

  0%|          | 0/50 [00:00<?, ?it/s]


IndexError: list index out of range

In [10]:
peti_df = pd.concat([peti_df_1, peti_df_2, peti_df_3, peti_df_4, peti_df_5, peti_df_6])
len(peti_df)

NameError: name 'peti_df_2' is not defined

In [ ]:
peti_df.head()

In [ ]:
peti_df.to_csv('blue-house-petition-best.csv', sep=',', encoding='utf-8')

In [ ]:
import pandas as pd 

petition = pd.read_csv('blue-house-petition-best.csv', thousands=',', encoding='utf-8', index_col=0)
petition.head()

In [ ]:
import re 
import time
from tqdm import tqdm
from urllib.request import urlopen
from bs4 import BeautifulSoup

content = []

for n in tqdm(petition.index): 
    url = urlopen(petition['URL'][n]) 
    if n / 2 == 0:
        time.sleep(1)
    else: 
        time.sleep(2)
    soup = BeautifulSoup(url, "html.parser") 
    
    # get petition content 
    content_raw = soup.find_all('div', class_='View_write')
    try: 
        if len(content_raw) > 1:
            content_raw_text = content_raw[1].get_text()
        else: 
            content_raw_text = content_raw[0].get_text()   
    except IndexError: 
        print('IndexError:', petition['URL'][n])
    
    content_split = re.split('\r|\t|\n|\xa0', content_raw_text)
    content.append(''.join(content_split))


In [ ]:
petition['청원내용'] = content

In [ ]:
petition.to_csv('final-blue-house-petition-best.csv', sep=',', encoding='utf-8')